In [1]:
# %pip install -q openai anthropic peft sentencepiece protobuf

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3,4,5'

In [3]:
import json
from tqdm.auto import tqdm

In [4]:
import sys
sys.path.append('/home/majed_alshaibani/Projects/jrcai_corekit/src')

In [5]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # suppress requests warnings of InsecureRequestWarning

In [6]:
from llm import *


/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [7]:
llm_loader = LLMLoader(
    '/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/',
    llm_initializer=Llama31Initializer(),
    )
messsage_generator = MessageGeneratorFromLocalLLM(llm_loader)

# Exploring Jais outputs

In [8]:
def get_chat_completion(messages):
    llm_response = messsage_generator(messages)
    return llm_response[0]

test the API

In [9]:
print(get_chat_completion(
    messages=[[{
            'role': 'user',
            'content': 'ما هو لون نجوم السماء؟'
        }]]
    )
)

loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/config.json
Model config LlamaConfig {
  "_name_or_path": "/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.43.3",
  "use_cac

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate 

Meta-Llama-3.1-70B-Instruct - batches sizes = [1]
النجوم في السماء بيضاء.


# Exploring the dataset

In [10]:
import datasets

In [11]:
dataset = datasets.load_dataset('arbml/AraSum')
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'summary', 'article'],
        num_rows: 49603
    })
})

In [12]:
articles = dataset['train']['article']
summaries = dataset['train']['summary']
len(articles), len(summaries)

(49603, 49603)

In [13]:
# max words, min words, avg words for summaries
max_summaries_words = max(len(text.split()) for text in summaries)
min_summaries_words = min(len(text.split()) for text in summaries)
avg_summaries_words = sum(len(text.split()) for text in summaries) / len(summaries)
max_summaries_words, min_summaries_words, avg_summaries_words

(45, 2, 33.53730621131786)

In [14]:
# max words, min words, avg words for articles
max_articles_words = max(len(text.split()) for text in articles)
min_articles_words = min(len(text.split()) for text in articles)
avg_articles_words = sum(len(text.split()) for text in articles) / len(articles)
max_articles_words, min_articles_words, avg_articles_words

(2898, 33, 376.54426143580025)

In [15]:
considered_articles,considered_summaries = [],[]
for article,summary in zip(articles,summaries):
    if 100 < len(article.split()) < 500 and 20 < len(summary.split()) < 50:
        considered_articles.append(article)
        considered_summaries.append(summary)
len(considered_articles),len(considered_summaries)

(36873, 36873)

In [16]:
articles = considered_articles[:3000]
summaries = considered_summaries[:3000]
len(articles),len(summaries)

(3000, 3000)

# Generating the dataset

first, we generate articles by polishing them.

In [17]:
def save_articles_to_jsonl(articles, file_path):  
    # Append new articles
    with open(file_path, 'w', encoding='utf-8') as f:
        for article in articles:
            json_object = article
            f.write(json.dumps(json_object, ensure_ascii=False) + '\n')

def load_articles_from_jsonl(file_path):
    articles = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                articles.append(json.loads(line))
    return articles

In [18]:
BATCH_SIZE = 100

articles_by_polishing_prompts = []

for article in articles:
    prompt = f"""
قم بإعادة صياغة هذه المقالة التالية، مراعيا تدقيقها لغويا ونحويا وبنائيا ومعنى، المقالة كتبت لتنشر في الصحافة.
قم بإعادة الصياغة فقط دون إضافة أي تعبيرات أخرى قبل إعادة الصياغة أو بعدها
المقالة:
{article}
    """.strip()
    articles_by_polishing_prompts.append(prompt)

# Create batches of messages
batched_messages = [
    [
        [{'role': 'user', 'content': prompt}] 
        for prompt in articles_by_polishing_prompts[i:i+BATCH_SIZE]
    ]
    for i in range(0, len(articles_by_polishing_prompts), BATCH_SIZE)
]

# Main process
!mkdir -p generated_arabic_datasets/llama-batched/arasum
file_path = "generated_arabic_datasets/llama-batched/arasum/generated_articles_from_polishing.jsonl"

generated_articles_from_polishing = []

for batch_idx, batch in enumerate(tqdm(batched_messages, desc="Processing batches")):
    generated_batch = messsage_generator(batch)
    
    batch_start = batch_idx * BATCH_SIZE
    batch_end = min((batch_idx + 1) * BATCH_SIZE, len(articles))
    
    batch_results = [
        {
            "original_article": article,
            "original_article_summary": summary,
            "generated_article": generated_article,
        }
        for article, summary, generated_article in zip(
            articles[batch_start:batch_end],
            summaries[batch_start:batch_end],
            generated_batch
        )
    ]
    
    generated_articles_from_polishing.extend(batch_results)
    
    # Save intermediate results after each batch
    save_articles_to_jsonl(generated_articles_from_polishing, file_path)

# Final save (this is actually redundant now, but kept for clarity)
save_articles_to_jsonl(generated_articles_from_polishing, file_path)

print(f"All batches processed and saved to {file_path}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 9]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 2]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 6]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 10, 10]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 10, 10, 1]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 4]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 9]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 10, 10]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 9]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 4]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 10, 10]
Meta-Llama-3.1-70B-Instruct - batches sizes = [7, 7, 7